## Импорт необходимых библиотек

In [1]:
!pip install tweet-preprocessor gensim kaggle sklearn

    ERROR: Command errored out with exit status 1:
     command: 'C:\ProgramData\Anaconda3\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\User\\AppData\\Local\\Temp\\pip-install-51yh4ys4\\sklearn\\setup.py'"'"'; __file__='"'"'C:\\Users\\User\\AppData\\Local\\Temp\\pip-install-51yh4ys4\\sklearn\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\User\AppData\Local\Temp\pip-install-51yh4ys4\sklearn\pip-egg-info'
         cwd: C:\Users\User\AppData\Local\Temp\pip-install-51yh4ys4\sklearn\
    Complete output (15 lines):
    The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
    rather than 'sklearn' for pip commands.
    
    Here is how to fix this error in the main use cases:
    - use 'pip install scikit-learn' rather than 'pip install sklearn'
    - replace 'sklearn' by 'scikit-learn' in you

In [2]:
import pandas as pd
import matplotlib
import preprocessor as p
from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS
import gensim
from sklearn.model_selection import train_test_split
from kaggle.api.kaggle_api_extended import KaggleApi
import zipfile

ModuleNotFoundError: No module named 'preprocessor'

## Анализ данных

In [ ]:
api = KaggleApi()
api.authenticate()
api.dataset_download_files('kazanova/sentiment140')

In [ ]:
with zipfile.ZipFile('sentiment140.zip', 'r') as zipref:
    zipref.extractall('sentiment140/')

In [ ]:
colnames=['target', 'ids', 'date', 'flag', 'user', 'text']
data = pd.read_csv("sentiment140/training.1600000.processed.noemoticon.csv", encoding='cp1252', names=colnames, header=None)
data.sample(10)

In [ ]:
data.info()

In [ ]:
data.drop(['ids', 'date', 'flag', 'user'], axis=1, inplace=True)
data.head()

In [ ]:
data['target'].value_counts().plot(
    kind='bar',
    rot=0,
    title='Количество твитов по классам',
    xlabel='Класс (0 - отрицательные, 2 - нейтральные, 4 - позитивные',
    ylabel='Число твитов',
    legend=True,
    grid=True
);

Класса получается всего 2, поэтому мы будем делать бинарную классификацию. Заменим метку класса 4 на 1:

In [ ]:
data['target'] = data['target'].replace(4,1)

## Предобработка данных

In [ ]:
print(STOPWORDS)

Данные стоп слова вряд ли влияют на настроение твита, так что возьму их прямо из коробки.

In [ ]:
%%time
def preprocess_tweet(row):
    text = row['text']
    text = p.clean(text)
    text = remove_stopwords(text)
    return text.lower().replace('\s\s+', ' ')

data['cleaned_text'] = data.apply(preprocess_tweet, axis=1)

In [ ]:
data.sample(10)

## Извлечение признаков из текста с помощью векторизации

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data['cleaned_text'], data['target'], random_state=2023)

In [ ]:
%%time
VECTOR_SIZE = 256
model = gensim.models.FastText(
    sentences=X_train,
    vector_size=VECTOR_SIZE, # default = 100
    window=7, # default = 5
    min_count=10,
    sg=1, # Training algorithm: 1 for skip-gram; otherwise CBOW
    hs=0, #  If 1, hierarchical softmax will be used for model training. If 0, and negative is non-zero, negative sampling will be used.
    negative=5, # If > 0, negative sampling will be used, if set to 0, no negative sampling is used.
    epochs=25, # Number of iterations (epochs) over the corpus
    seed=2023,
)